# Training simplistic models
The exact conditions for protein crystallization include pH, Temperature, protein concentration, physical setup and buffer ingredients and concentrations. 
A proper combination of all of the above will result in crystallization, but the space of possibilities for conditions is immense.
As a POC, we will train a simple model to predict crystallization conditions separately.  

Important: (This is done for POC, While the biochemical meaning of predicting temperature or pH by themselves without the buffer is very limited.)

In [4]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import re

VEC_COLS = list(np.arange(1280))
ENTRY_COLS = ['pdb_id', 'struct_method', 'crystal_method',             'ph',           'temp',
         'pdbx_details',   'deposit_date', 'revision_date']


In [5]:
#load the pdb data
df = pd.read_csv('../data/pdb_data/processed_data.csv')
df.shape

#Load the ESM vectors were generated using crystoper/vectorize.py
vec_path = '../vectors/sequences/esm2.npy'
vectors = pd.DataFrame(np.load(vec_path))

In [6]:
df

,pdb_id,pe_index,sequence,poly_type,struct_method,crystal_method,ph,temp,pdbx_details,deposit_date,revision_date
0,5IQ6,1,MDVIGERIKRIKEEHNSTWHYDDENPYKTWAYHGSYEVKATGSASS...,Protein,X-RAY DIFFRACTION,LCP,7.5,293.0,"HEPES pH7.5, PAA (poly acrilic acid) 20%, MgCl...",2016-03-10T00:00:00+0000,2024-05-08T00:00:00+0000
1,5IQB,1,MEYRYDDNATNVKAMKYLIEHYFDNFKVDSIEIIGSGYDSVAYLVN...,Protein,X-RAY DIFFRACTION,LCP,7.5,296.0,"80-120mM MgCl2, 8% glycerol, 10% PEG 3350, 100...",2016-03-10T00:00:00+0000,2023-09-27T00:00:00+0000
2,5IQ7,3,RRRNEQELLELDKWASLWNWFDITNWLWYIRRRR,Protein,X-RAY DIFFRACTION,LCP,7.5,293.0,"40% PEG400, 0.1M NaCitrate, 0.1M Tris-HCl, 7.5",2016-03-10T00:00:00+0000,2023-09-27T00:00:00+0000
3,5IQ7,2,YELTQETGVSVALGRTVTITCRGDSLRSHYASWYQKKPGQAPILLF...,Protein,X-RAY DIFFRACTION,LCP,7.5,293.0,"40% PEG400, 0.1M NaCitrate, 0.1M Tris-HCl, 7.5",2016-03-10T00:00:00+0000,2023-09-27T00:00:00+0000
4,5IQ7,1,EVQLVESGGGLVKPGGSLRLSCSASGFDFDNAWMTWVRQPPGKGLE...,Protein,X-RAY DIFFRACTION,LCP,7.5,293.0,"40% PEG400, 0.1M NaCitrate, 0.1M Tris-HCl, 7.5",2016-03-10T00:00:00+0000,2023-09-27T00:00:00+0000
...,...,...,...,...,...,...,...,...,...,...,...
222361,1L9E,1,STHFDVIVVGAGSMGMAAGYQLAKQGVKTLLVDAFDPPHTNGSHHG...,Protein,X-RAY DIFFRACTION,LCP,7.0,295.0,"phosphate, imidazole, pH 7.0, VAPOR DIFFUSION,...",2002-03-22T00:00:00+0000,2023-08-16T00:00:00+0000
222362,6L98,1,YVLTQPPSVSVAPGKTARIACEGNNIGSKSVHWYQQKPGQAPVLVV...,Protein,X-RAY DIFFRACTION,LCP,6.5,283.0,100 mM MES\n15% glycerol\n15% PEG6000,2019-11-08T00:00:00+0000,2023-11-22T00:00:00+0000
222363,3L9V,1,SNAEWESITPPVVDAPAVVEFFSFYCPPCYAFSQTMGVDQAIRHVL...,Protein,X-RAY DIFFRACTION,LCP,7.0,293.0,"1.45M sodium malonate, 0.5% (w/v) Jeffamine ED...",2010-01-05T00:00:00+0000,2017-11-01T00:00:00+0000
222364,7L9Q,1,GSHMAVQIGFLLFPEVQQLDLTGPHDVLASLPDVQVHLIWKEPGPV...,Protein,X-RAY DIFFRACTION,LCP,8.6,298.0,"23% PEG3350, 100 mM Tris-HCl, pH 8.6, 200 mM m...",2021-01-04T00:00:00+0000,2023-10-18T00:00:00+0000


In [7]:
def filter(df, vectors, mask):
    return df[mask].reset_index(drop=True), vectors[mask].reset_index(drop=True)

In [8]:
#concat vectors to df
df = pd.concat([df, vectors], axis=1)
del vectors

df_vecs = df[['pdb_id'] + VEC_COLS]
df = df[ENTRY_COLS]

In [9]:
#count entities, avarage vectors and create a new df
avg = df_vecs.groupby('pdb_id')[VEC_COLS].sum().reset_index()

counts = df_vecs[['pdb_id']].groupby('pdb_id').size().reset_index()
counts.columns = ['pdb_id', 'n_entities']

df = df[ENTRY_COLS].drop_duplicates().merge(counts, how='left').merge(avg, how='left')

del avg
del counts


 # pH predicting model    

In [15]:
X = df[VEC_COLS]
y = df.ph

AttributeError: 'DataFrame' object has no attribute 'pH'